In [1]:
!pwd

/Users/sofiepelvig/apartments/notebooks


In [2]:
%cd ..

/Users/sofiepelvig/apartments


In [3]:
from ipyleaflet import (Map, basemaps, basemap_to_tiles, 
                        GeoData, Marker, MarkerCluster)

import pandas as pd
import geopandas
import matplotlib.pyplot as plt
from geodatasets import get_path

In [4]:
df = pd.read_csv("assets/data/bolig.csv.gzip", compression="gzip", parse_dates=["soldDate"])
df.tail(3)

,estateId,address,zipCode,price,soldDate,propertyType,saleType,sqmPrice,rooms,size,...,change,guid,latitude,longitude,municipalityCode,estateCode,city,groupKey,canGetVR,bfEnr
99208,0,"Hørsholmsgade 8, 4. tv",2200,11947874,1992-01-06 23:00:00+00:00,3,Alm. Salg,225431.580,2.0,53,...,0.0,DA7226E2-F5C9-43D1-AEE6-F928CE45212E,55.690900,12.543945,101,260348,København N,1.0,True,6019808
99209,0,"Peter Fabers Gade 41, 2. th",2200,250000,1992-01-03 23:00:00+00:00,3,Alm. Salg,5000.000,2.0,50,...,0.0,D9388837-4672-4B9A-A1FE-C65BB6397DD8,55.691830,12.556379,101,436128,København N,NaN,True,146895
99210,0,"Øresundsvej 108, 1. tv",2300,351000,1992-01-02 23:00:00+00:00,3,Alm. Salg,5754.098,2.0,61,...,0.0,B18BD911-7FF3-4977-A46D-D324E247A543,55.661255,12.621660,101,661318,København S,NaN,True,133305


In [5]:
alm_salg_df = df[df['saleType'].eq('Alm. Salg')]

In [ ]:
%%time

# SAMPLE =20_000
YEAR = 2008  # greater or equal to <=
markers = (
    alm_salg_df
     .loc[lambda d: d["soldDate"].dt.year.ge(YEAR)]
     .assign(location=lambda d: d[["latitude", "longitude"]].agg(
         lambda d: (d["latitude"], d["longitude"]) ,axis=1),  # (latitude, longitude)
            title = lambda d: d[["address", "zipCode"]].agg(
         lambda d: f'{d["address"]}, {d["zipCode"]}' ,axis=1), # title will be address
          alt = (lambda d: d[["price", "size", "rooms" , "buildYear"]].agg(
         lambda d: f'price: {d["price"]}, size: {d["size"]}, rooms: {d["rooms"]}, build year: {d["buildYear"]}' ,axis=1))  # title will be address
    )
    .loc[:, ["location", "title", "alt"]]
    # .sample(SAMPLE)
    .agg(lambda d: Marker(location=d["location"], title=d["title"], alt=d["alt"]), axis=1)
)

In [7]:
markers.shape[0]

76898

In [8]:
#creating the map and centering it at sofie
# 55.692205626007144, 12.545636686596445
m = Map(center=(55.69220562, 12.5456366), zoom=15)

In [9]:
#adding the markers as a cluster 
marker_cluster = MarkerCluster(
    markers=markers.tolist()
)

In [10]:
# Add the GeoData layer to the map
m.add_layer(marker_cluster)

# Display the map
m

Map(center=[55.69220562, 12.5456366], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

In [ ]:
1 + 1

In [ ]:
markers.isna().sum()